In [ ]:
import numpy as np
import pandas as pd
from tqdm import tqdm
import matplotlib.pyplot as plt
import os
import sys
import tifffile
import imageio
import glob
import shutil
import glob
import shutil
import time
import cv2

In [ ]:
!pip install pycocotools

In [ ]:
# Original Image Dimensions
HEIGHT = 520
WIDTH = 704
SHAPE = (HEIGHT, WIDTH)

In [ ]:
def rles_to_mask(encs, shape=SHAPE):
    """
    Decodes a rle.

    Args:
        encs (list of str): Rles for each class.
        shape (tuple [2]): Mask size.

    Returns:
        np array [shape]: Mask.
    """
    img = np.zeros(shape[0] * shape[1], dtype=np.uint)
    if type(encs)==float:
        return img
    for m, enc in enumerate(encs):
        if isinstance(enc, np.float) and np.isnan(enc):
            continue
        enc_split = enc.split()
        for i in range(len(enc_split) // 2):
            start = int(enc_split[2 * i]) - 1
            length = int(enc_split[2 * i + 1])
            img[start: start + length] = 1 + m
    return img.reshape(shape)

## <font color='emperal'/> **Prepare LIVECell data**

In [ ]:
import sys
sys.path.append('../input/sartorius-helper-package')
from livecell_dataset import LiveCell_Dataset

In [ ]:
# Define path constances
SHSY5Y_LIVECELL_TRAIN_IMAGE_FOLDER = '../input/sartorius-cell-instance-segmentation/LIVECell_dataset_2021/images/livecell_train_val_images/SHSY5Y'
SHSY5Y_LIVECELL_TRAIN_ANNOTATION_PATH = '../input/sartorius-cell-instance-segmentation/LIVECell_dataset_2021/annotations/LIVECell_single_cells/shsy5y/livecell_shsy5y_train.json'
SHSY5Y_LIVECELL_VAL_ANNOTATION_PATH = '../input/sartorius-cell-instance-segmentation/LIVECell_dataset_2021/annotations/LIVECell_single_cells/shsy5y/livecell_shsy5y_val.json'

In [ ]:
!mkdir ./train ./val

In [ ]:
livecell_dataset = LiveCell_Dataset(image_folder=SHSY5Y_LIVECELL_TRAIN_IMAGE_FOLDER, 
                                    train_annotation_path=SHSY5Y_LIVECELL_TRAIN_ANNOTATION_PATH,
                                    val_annotation_path=SHSY5Y_LIVECELL_VAL_ANNOTATION_PATH,
                                    shape_target=SHAPE,
                                    shape=SHAPE)

# Convert polygon mask to rle
livecell_dataset.convert_polygon_to_rle()

# Save images to working dir
livecell_dataset.save_image_to_working_dir('train', pad_img=False)
livecell_dataset.save_image_to_working_dir('val', pad_img=False)

# Get LiveCell DataFrame
livecell_train_df = livecell_dataset.create_livecell_dataframe(img_group='train')
livecell_val_df = livecell_dataset.create_livecell_dataframe(img_group='val')

In [ ]:
!zip -r -q livecell_train.zip train
!zip -r -q livecell_val.zip val

In [ ]:
#livecell_train_df = livecell_train_df.groupby('id')['annotation'].agg(list).reset_index()
#livecell_val_df = livecell_val_df.groupby('id')['annotation'].agg(list).reset_index()

In [ ]:
livecell_train_df

In [ ]:
livecell_train_df.to_csv('./livecell_train.csv', index=False)

In [ ]:
livecell_val_df

In [ ]:
livecell_val_df.to_csv('./livecell_val.csv', index=False)